In [1]:
import cirq
from cirq import Simulator
import numpy as np

In [2]:
class SqrtISWAP(cirq.Gate):
    def __init__(self):
        super(SqrtISWAP, self)

    def _num_qubits_(self):
        return 2
    
    # Obtained from: https://github.com/quantumlib/Cirq/blob/a22269dfe41b0da78243bbd210a915d26cc7d25f/cirq-core/cirq/ops/swap_gates.py#L166
    def _decompose_(self, qubits):
        q0, q1 = qubits
        yield cirq.CX(q0,q1)
        yield cirq.H(q0)
        yield cirq.CX(q1,q0)
        yield cirq.ZPowGate(exponent=0.25).on(q0)
        yield cirq.CX(q1,q0)
        yield cirq.ZPowGate(exponent=-0.25).on(q0)
        yield cirq.H(q0)
        yield cirq.CX(q0,q1)
        
    def _circuit_diagram_info_(self, args):
        return ["sqrt(iSWAP)"] * self.num_qubits()

In [3]:
class IonSupportedGivens(cirq.Gate):
    def __init__(self, theta):
        super(IonSupportedGivens, self)
        self.theta = theta

    def _num_qubits_(self):
        return 2

    def _decompose_(self, qubits):
        
        # Have to EXPLICITLY call decomposition, Azure Quantum doesn't seem to auto-decompose
        # to IonQ supported gates
        q0, q1 = qubits
        yield SqrtISWAP().on(q0,q1)._decompose_()
        
        yield cirq.Rz(rads= -self.theta).on(q0)
        yield cirq.Rz(rads=self.theta + np.pi).on(q1)

        yield SqrtISWAP().on(q0,q1)._decompose_()
        yield cirq.Rz(rads=np.pi).on(q1)
        
    def _circuit_diagram_info_(self, args):
        return ["IonGivens({:.3f})".format(self.theta)] * self.num_qubits()

In [4]:
# single qubit test
rot_angle = np.pi / 3

circuit = cirq.Circuit()
q0, q1 = cirq.LineQubit.range(2)
circuit.append(IonSupportedGivens(theta=rot_angle).on(q0,q1))
print(circuit)

0: ───IonGivens(1.047)───
      │
1: ───IonGivens(1.047)───


In [5]:
# see the matrix unitary
# single qubit test
circuit.unitary()

array([[-1.       -1.34015774e-16j,  0.       +0.00000000e+00j,
         0.       +0.00000000e+00j,  0.       -3.72797834e-17j],
       [ 0.       +0.00000000e+00j, -0.5      -5.88784672e-17j,
         0.8660254+1.17756934e-16j,  0.       +0.00000000e+00j],
       [ 0.       +0.00000000e+00j, -0.8660254+5.88784672e-17j,
        -0.5      +1.96261557e-16j,  0.       +0.00000000e+00j],
       [ 0.       -2.29934717e-17j,  0.       -0.00000000e+00j,
         0.       -0.00000000e+00j, -1.       +3.51298275e-16j]])

In [6]:
simulator = Simulator()

# Cirq state initialization tip from Craig Gidney: https://quantumcomputing.stackexchange.com/a/5496
# Some simple initial amplitudes that can be verified by hand
init = [1/np.sqrt(2), 1/np.sqrt(2), 0, 0]
full_superposition = [1/np.sqrt(4)] * 4
result = simulator.simulate(circuit, qubit_order=[q0, q1], initial_state=full_superposition)

In [7]:
np.round(result.final_state_vector, 3)

array([-0.5  -0.j,  0.183+0.j, -0.683-0.j, -0.5  +0.j], dtype=complex64)

In [8]:
result.final_state_vector

array([-0.49999994-0.0000000e+00j,  0.1830127 +2.1073424e-08j,
       -0.68301266-2.1073424e-08j, -0.49999994+0.0000000e+00j],
      dtype=complex64)

In [9]:
# create a circuit for testing purposes

## Single Excitation circuit from: https://pennylane.ai/qml/demos/tutorial_givens_rotations.html
## Create state |100>
## Apply givens between qubits [0,1], then [0,2] 
## with theta =−2arcsin(1/sqrt(3)),phi =−2arcsin(1/sqrt(2)).

x = -2 * np.arcsin(np.sqrt(1/3))
y = -2 * np.arcsin(np.sqrt(1/2))

circuit = cirq.Circuit()
q0, q1, q2 = cirq.LineQubit.range(3)
circuit.append(cirq.X(q0))
circuit.append(IonSupportedGivens(theta=x/2).on(q0,q1))
circuit.append(IonSupportedGivens(theta=y/2).on(q0,q2))
print(circuit)

0: ───X───IonGivens(-0.615)───IonGivens(-0.785)───
          │                   │
1: ───────IonGivens(-0.615)───┼───────────────────
                              │
2: ───────────────────────────IonGivens(-0.785)───


In [10]:
simulator = Simulator()

In [11]:
result = simulator.simulate(circuit, qubit_order=[q0, q1, q2])

In [12]:
np.round(result.final_state_vector, 3)

array([0.   +0.j, 0.577-0.j, 0.577-0.j, 0.   +0.j, 0.577-0.j, 0.   +0.j,
       0.   +0.j, 0.   +0.j], dtype=complex64)

In [13]:
for ind, i in enumerate(result.final_state_vector):
    print(ind, i)

0 0j
1 (0.57735014-4.2146848e-08j)
2 (0.57735026-8.6513546e-10j)
3 0j
4 (0.5773502-1.2560739e-15j)
5 0j
6 0j
7 8.6513546e-10j
